In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [16]:
    england = ["https://www.transfermarkt.co.uk/fc-liverpool/startseite/verein/31/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/manchester-city/startseite/verein/281/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/fc-chelsea/startseite/verein/631/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/tottenham-hotspur/startseite/verein/148/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/manchester-united/startseite/verein/985/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/fc-arsenal/startseite/verein/11/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/fc-everton/startseite/verein/29/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/leicester-city/startseite/verein/1003/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/wolverhampton-wanderers/startseite/verein/543/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/west-ham-united/startseite/verein/379/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/aston-villa/startseite/verein/405/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/newcastle-united/startseite/verein/762/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/fc-fulham/startseite/verein/931/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/crystal-palace/startseite/verein/873/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/fc-southampton/startseite/verein/180/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/brighton-amp-hove-albion/startseite/verein/1237/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/leeds-united/startseite/verein/399/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/fc-burnley/startseite/verein/1132/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/sheffield-united/startseite/verein/350/saison_id/2021/plus/1",
              "https://www.transfermarkt.co.uk/west-bromwich-albion/startseite/verein/984/saison_id/2021/plus/1"]
    

In [17]:
#Creating a function that takes a list of links as an argument. 
#The function will scrape through the provided links' corresponding webpages and return a dataframe of the necessary scraped player data.
def transfermarkt(link_list):

    headers = {'User-Agent': 
                   'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    club_links = link_list

    for a in club_links:

        page = a
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Players = pageSoup.find_all("img", {"class": "bilderrahmen-fixed lazy lazy"})
        Age = pageSoup.find_all("td", {"class": "zentriert"})
        Values = pageSoup.find_all("td", {"class": "rechts hauptlink"})
        Positions = pageSoup.find_all("td", {"class": ["zentriert rueckennummer bg_Torwart","zentriert rueckennummer bg_Abwehr","zentriert rueckennummer bg_Mittelfeld","zentriert rueckennummer bg_Sturm"]})
        Nationality = pageSoup.find_all("td", {"class": "zentriert"})
        Club = str(pageSoup.find_all("meta")[10])[15:].split(" - Club profile",1)[0]
        League = str(pageSoup.find_all("meta")[5]).split(',')[1]


        if a == club_links[0]:
            PlayersList = []
            AgeList=[]
            NationList=[]
            ValuesList = []
            PositionsList=[]
            ClubList = []
            LeagueList = []

            for i in range(0,len(Values)):
                ValuesList.append(Values[i].text)

            for i in range(0,len(Players)):
                PlayersList.append(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1])

            for i in range(0,len(Positions)):
                PositionsList.append(str(Positions[i]).split('title="',1)[1].split('"><div')[0])

            for i in range(1,(len(Players)*3),3):
                AgeList.append(str(Age[i]).split("(",1)[1].split(")",1)[0])

            for i in range(2,(len(Players)*3),3):
                NationList.append(str(Nationality[i]).split('title="',1)[1].split('"/',1)[0])
                
            for i in range(0,len(Players)):
                ClubList.append(str(pageSoup.find_all("meta")[10])[15:].split(" - Club profile",1)[0])
                
            for i in range(0,len(Players)):
                LeagueList.append(League)
            

        else:

            for i in range(0,len(Values)):
                ValuesList.append(Values[i].text)

            for i in range(0,len(Players)):
                PlayersList.append(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1])

            for i in range(0,len(Positions)):
                PositionsList.append(str(Positions[i]).split('title="',1)[1].split('"><div')[0])

            for i in range(1,(len(Players)*3),3):
                AgeList.append(str(Age[i]).split("(",1)[1].split(")",1)[0])

            for i in range(2,(len(Players)*3),3):
                NationList.append(str(Nationality[i]).split('title="',1)[1].split('"/',1)[0])
            
            for i in range(0,len(Players)):
                ClubList.append(str(pageSoup.find_all("meta")[10])[15:].split(" - Club profile",1)[0])
            
            for i in range(0,len(Players)):
                LeagueList.append(League)


    #Initial uncleaned dataframe initiated
    df= pd.DataFrame({"Players":PlayersList,"Club":ClubList,"League":LeagueList,"Position":PositionsList, "Age":AgeList, "Nationality":NationList,"Values":ValuesList,})
    
    #Missing Transfer Values were stored as '\xa0'. The line below replaces them as None values
    df.loc[(df.Values == '\xa0'),'Values']= None

    #Dataframe without missing transfer values
    df_droppednull = df.dropna()

    #Converting the Transfer Values data to float            
    transfer_values = [a[1:len(a)-2] for a in df_droppednull['Values']]

    cleaned_values=[]
    for a in list(range(len(transfer_values))):
        if transfer_values[a].endswith('m'):
            cleaned_values.append(float(transfer_values[a][:len(transfer_values[a])-1])*1000000)
        elif transfer_values[a].endswith('Th.'):
            cleaned_values.append(float(transfer_values[a][:len(transfer_values[a])-3])*1000)
        else:
            cleaned_values.append

    #Constructing the Final Dataframe   
    final_df= pd.DataFrame({"Player":df_droppednull['Players'],"Club":df_droppednull['Club'],"League":df_droppednull['League'],
                            "Age":df_droppednull['Age'],"Position":df_droppednull['Position'],
                            "Nation":df_droppednull['Nationality'],"Value":cleaned_values})

    #df ranked by transfer value
    ranked_df = final_df.sort_values('Value',ascending=False)
    return ranked_df

In [18]:
transfermarkt(england)

,Player,Club,League,Age,Position,Nation,Value
119,Harry Kane,Tottenham Hotspur,Premier League,28,attack,England,108000000.0
90,Romelu Lukaku,Chelsea FC,Premier League,28,attack,Belgium,90000000.0
145,Jadon Sancho,Manchester United,Premier League,21,attack,England,90000000.0
26,Mohamed Salah,Liverpool FC,Premier League,29,attack,Egypt,90000000.0
48,Kevin De Bruyne,Manchester City,Premier League,30,midfield,Belgium,90000000.0
...,...,...,...,...,...,...,...
390,Harry Lewis,Southampton FC,Premier League,23,Goalkeeper,England,180000.0
507,Iliman Ndiaye,Sheffield United,Championship,21,midfield,France,180000.0
458,Jordan Stevens,Leeds United,Premier League,21,midfield,England,180000.0
353,Tyrese Francois,Fulham FC,Championship,21,midfield,Australia,135000.0
